**Task.** Query for PowerShell abuse using PowerShell.  

**Purpose.** PowerShell is a command-line and scripting interface within the Windows operating system. Adversaries abuse PowerShell to download and execute malicious code.  

**Conditions.** PowerShell Remoting and Script Block Logging is enabled.  

**Standard.** You were able to perform a query and determine if suspicious or malicious activity occurred.

In [ ]:
$ComputerName = "localhost"
$FilterHashTable = @{
    LogName = "Microsoft-Windows-PowerShell/Operational"
    Id = 4104
}

filter Read-WinEvent {
    $WinEvent = [ordered]@{} 
    $XmlData = [xml]$_.ToXml()
    $SystemData = $XmlData.Event.System
    $SystemData | 
    Get-Member -MemberType Properties | 
    Select-Object -ExpandProperty Name |
    ForEach-Object {
        $Field = $_
        if ($Field -eq 'TimeCreated') {
            $WinEvent.$Field = Get-Date -Format 'yyyy-MM-dd hh:mm:ss' $SystemData[$Field].SystemTime
        } elseif ($SystemData[$Field].'#text') {
            $WinEvent.$Field = $SystemData[$Field].'#text'
        } else {
            $SystemData[$Field]  | 
            Get-Member -MemberType Properties | 
            Select-Object -ExpandProperty Name |
            ForEach-Object { 
                $WinEvent.$Field = @{}
                $WinEvent.$Field.$_ = $SystemData[$Field].$_
            }
        }
    }
    $XmlData.Event.EventData.Data |
    ForEach-Object { 
        $WinEvent.$($_.Name) = $_.'#text'
    }
    return New-Object -TypeName PSObject -Property $WinEvent
}

Get-WinEvent -ComputerName $ComputerName -FilterHashTable $FilterHashTable |
Read-WinEvent | 
Select-Object -First 1


   TypeName: Selected.System.Management.Automation.PSCustomObject

Name            MemberType   Definition
----            ----------   ----------
Equals          Method       bool Equals(System.Object obj)
GetHashCode     Method       int GetHashCode()
GetType         Method       type GetType()
ToString        Method       string ToString()
Channel         NoteProperty string Channel=Microsoft-Windows-PowerShell/Operational
Computer        NoteProperty string Computer=LAPTOP-H4TI33TA
Correlation     NoteProperty hashtable Correlation=System.Collections.Hashtable
EventID         NoteProperty string EventID=4104
EventRecordID   NoteProperty string EventRecordID=210511
Execution       NoteProperty hashtable Execution=System.Collections.Hashtable
Keywords        NoteProperty string Keywords=0x0
Level           NoteProperty string Level=5
MessageNumber   NoteProperty string MessageNumber=1
MessageTotal    NoteProperty string MessageTotal=1
Opcode          NoteProperty string Opcode=15
Pa